In [1]:
import numpy


In [8]:
# mesh paramters
nx = 81
dx = 0.25
dt = 0.002
gamma = 1.4

x = numpy.linspace(-10,10,nx)
# initial conditions


In [22]:
# functions
# initial condition
def init_cond(nx):
    """
    Creates initial conditions for shock tube problem
    """
    # initialize rho
    rho_init = numpy.zeros(nx)
    rho_init[:int((nx-1)/2)] = 1. # left side density = 1 kg/m^3
    rho_init[int((nx-1)/2):] = 0.125 # right side density = 0.125 kg/m&3
    
    # initialize u
    u_init = numpy.zeros(nx)
    
    #initialize pressure
    p_init = numpy.zeros(nx)
    p_init[:int((nx-1)/2)] = 100e3 # left side pressure = 100k Pa
    p_init[int((nx-1)/2):] = 10e3  # right side pressure = 10k Pa
    return rho_init, p_init, u_init

# flux
def computeF(u):
    u1 = u[0]
    u2 = u[1]
    u3 = u[2]
    return [u2,
            u2**2/2 + (gamma-1)*(u3 - 1/2*u2**2/u1),
            (u3 + (gamma - 1)*(u3 - 1/2*u2**2/u1))*u2/u1]

# Richtmyer method
def richtmyer():
    
    return

In [21]:
x[:int((nx-1)/2)]

array([-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
        -8.  ,  -7.75,  -7.5 ,  -7.25,  -7.  ,  -6.75,  -6.5 ,  -6.25,
        -6.  ,  -5.75,  -5.5 ,  -5.25,  -5.  ,  -4.75,  -4.5 ,  -4.25,
        -4.  ,  -3.75,  -3.5 ,  -3.25,  -3.  ,  -2.75,  -2.5 ,  -2.25,
        -2.  ,  -1.75,  -1.5 ,  -1.25,  -1.  ,  -0.75,  -0.5 ,  -0.25])